#### Create and Train a RNN 
to predict the value of the voltage rise on the next period

Prediction is done in [VoltageRiseNum_Pred](VoltageRiseNum_Pred.ipynb)

We consider 

\begin{equation}
Z(k) = \begin{bmatrix} X(k-1)\\ \vdots \\ X(k-6)\ \end{bmatrix}, \;\; \text{where} \; \; 
X(k-i) = \begin{bmatrix} P_{load}(k-i) \\ P_{BT}(k-i) \\ P0013(k-i) \\ P0018(k-i) \\ P0100(k-i)\\ max\_vm\_pu(k-i) \end{bmatrix}
\end{equation}

to predict 

\begin{equation} Pred = \begin{bmatrix} P_{load}(k)\\  max\_vm\_pu(k) \end{bmatrix} \end{equation}



I am predicting column [0,5] i.e ['Cons','Voltage_rise'] even though the only column of interest is 5. 
This is because the RNN does not learn well when one is predicting on a unique feature. 
Column 5 is the one considered among all because it has the smoothest curve within the backward 
looking window considered hence yield the best accuracy

---

#### Import modules to be used

In [7]:
#Import Modules 
import pandas as pd
import pandapower as pp
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm # Profiling 
import seaborn as sbn
import pickle, sys, importlib,  time
import os
from pickle import load
import tensorflow as tf
import joblib

#### Import Module for ML

In [8]:
# import pakages forML
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_absolute_error, mean_squared_error

#### Import my own modules

In [9]:
# import psutil
function_folder = 'py_files/' 
# Add function folder to path if it is not already
if function_folder not in sys.path: sys.path.append(function_folder)

import oriFunctions as oriFc
from oriFunctions import network_folder, excel_folder, py_folder, Δt, attr_list

In [10]:
#Import Networks
net_civaux=pp.from_pickle(f'{network_folder}CIVAUX.p')
net_stlaurent=pp.from_pickle(f'{network_folder}ST LAURENT.p')

In [11]:
#Load files
file_p_inj_0013 = 'Prod_HTA/PROD_Bis/P0013/PROD-BATISOLAIRE 6-CIVAUX - Actif injecte (P-).csv'
file_p_inj_0018 = 'Prod_HTA/PROD_Bis/P0018/PROD-SUN POITOU 2516 (Z.I de la Pitage)-LHOMMAIZE - Actif injecte (P-).csv'


# The  commissioning of the Prod P0100 is recent (2022). I therefore use the data of the closer energy 
# producer that is P0058  and consider it as that of Prod P0100 
file_p_inj_0100 = 'Prod_HTA/PROD_Bis/PROD-SERGIES 2204 (LA ROCHE A CORNUCHON)-PINDRAY - Actif P-.csv'

file_prod_bt_total = 'PROD_BTSUP36_SAINT LAURENT.csv'
file_cons_total = 'CONSO_POSTE_SAINT LAURENT.csv'


# Get files data 
p_mw_0013 = oriFc.readAndReshape_excelFile(file_p_inj_0013 ,excel_folder, )
p_mw_0018 = oriFc.readAndReshape_excelFile(file_p_inj_0018 ,excel_folder,)
p_mw_0100 = oriFc.readAndReshape_excelFile(file_p_inj_0100 ,excel_folder,)


p_mw_prod_bt_total = oriFc.readAndReshape_excelFile(file_prod_bt_total, excel_folder)
p_mw_cons_total = oriFc.readAndReshape_excelFile(file_cons_total, excel_folder)

# Create dict for all HT producers
dict_prod_hv = {'P0013': p_mw_0013[:len(p_mw_0100)], 
                'P0018': p_mw_0018[:len(p_mw_0100)],
                'P0100': p_mw_0100[:len(p_mw_0100)]
               }
# Create index to use for dataframe
per_index = pd.period_range('01 01 2020', periods=len(p_mw_0100), freq='10T')

# Use the create dict to create a dataFrame for Prod P0100
df_prodP0100 = pd.DataFrame(p_mw_0100, index=per_index)

# Use the create dict to create a dataFrame for all HT producers
df_prodHT = pd.DataFrame(dict_prod_hv, index=per_index)

# Dataframe prod BT 
per_index = pd.period_range('01 01 2020', periods=len(p_mw_prod_bt_total), freq='10T')
df_prod_bt_total = pd.DataFrame(p_mw_prod_bt_total, index=per_index, columns=['Prod_BT'])


# Dataframe Conso BT 
per_index = pd.period_range('01 01 2020', periods=len(p_mw_cons_total), freq='10T')
df_cons_total = pd.DataFrame(p_mw_cons_total, index=per_index, columns=['Cons'])

# Data cleaning on Consumption
# Replacing wrong data by the mean of surrounding periods
# 2020 -------------------------
previous_days = df_cons_total[(per_index>='2020 06 11') & (per_index<='2020 06 21 23:50')]
following_days = df_cons_total[(per_index>='2020 07 03') & (per_index<='2020 07 13 23:50')]
# Put the interpolated data into the dataframe
df_cons_total[(per_index>='2020 06 22') & (per_index<='2020 07 02 23:50')] = (np.array(following_days) + 
                                                                              np.array(previous_days) )/2
# 2022 -------------------------
previous_days = df_cons_total[(per_index>='2022 02 12') & (per_index<='2022 02 21 23:50')]
following_days = df_cons_total[(per_index>='2022 03 03') & (per_index<='2022 03 12 23:50')]
# # Put the interpolated data into the dataframe
df_cons_total[(per_index>='2022 02 21') & (per_index<='2022 03 02 23:50')] = (np.array(following_days) + 
                                                                              np.array(previous_days) )/2
# Dataframe PROD HT
# Replacing missing data by the mean of surrondings periods
# Extract previous and following days 
per_index = df_prodHT.index
previous_days = df_prodHT.loc[(per_index>='2020 06 11') & (per_index<='2020 06 21 23:50'),['P0013','P0018']]
following_days = df_prodHT.loc[(per_index>='2020 07 03') & (per_index<='2020 07 13 23:50'),['P0013','P0018']]

# Put the interpolated data into the dataframe
df_prodHT.loc[(per_index>='2020 06 22') & (per_index<='2020 07 02 23:50'), ['P0013','P0018']] = (np.array(following_days) +
                                                                                                 np.array(previous_days) ) /2


# Get total Power of BT producers
# Bt producers are indexed by the name None
max_p_mw_total_prodBT = net_civaux.sgen.max_p_mw[net_civaux.sgen.name.isna()].sum()

# # Get total power of load in the network
max_p_mw_total_load = net_civaux.load.max_p_mw.sum()

# Select relevant data up to 2022 06 01
df_prodHT = df_prodHT[df_prodHT.index<='2022 06 01 23:50']
df_prod_bt_total = df_prod_bt_total[df_prod_bt_total.index<='2022 06 01 23:50']
df_cons_total = df_cons_total[df_cons_total.index<='2022 06 01 23:50']


# Extract only dailight period i.e. from 07am to 7PM
# The daylight period is considered to be defined betwenn 07am and 7Pm excluded. 
h_start_end = ('06:50','18:50') # for the persistance model, the previous period i.e. 06:50 
                                # is needed to compute the first instant i.e. 07:00
per_index = df_prodHT.index
per_daylight = ( pd.Series(index=per_index.to_timestamp(), dtype=object).between_time(*h_start_end) ).index.to_period('10T')
day_tot_per = len(per_daylight[(per_daylight.year==2020)&(per_daylight.month==1)&(per_daylight.day==1)])


# Put all the data in a unique dataframe
df_data = pd.concat([df_cons_total, df_prod_bt_total, df_prodHT], axis=1).loc[per_daylight]

# # Extract only the relavant testing set since the training set covers the first part of the data
df_final = df_data[df_data.index<'2021 06 01']
per_index = df_final.index
per_index2 = ( pd.Series(index=per_index.to_timestamp(), dtype=object
                           ).between_time('07:00','18:50') ).index.to_period('10T')


vm_mu_max, vm_mu_min = 1.0250, 0.95  # Choosen 


#### Import the voltage rise from [Voltage_rise](VoltageRiseBinary.ipynb)

In [12]:
# Import the voltage rise from 
bin_volt_rise = joblib.load('pickle_files/simulationResults/Binary_Voltage_Rise.pkl')
df_final['Volt_Rise_bin'] = bin_volt_rise['Volt_Rise_Bin']

### Set variables For numerical voltage rise prediction

In [13]:
# # Extract only the relavant testing set since the training set covers the first part of the data
df_final = df_data[df_data.index<'2021 06 01']
per_index = df_final.index
per_index2 = ( pd.Series(index=per_index.to_timestamp(), dtype=object
                        ).between_time('07:00','18:50') ).index.to_period('10T')
df_final['Volt_Rise_num'] = bin_volt_rise['known']

In [17]:
df_train.head()

,Cons,Prod_BT,P0013,P0018,P0100,Volt_Rise_num
2020-01-01 06:50,11.489,0.0,0.0,0.0,0.0,1.0
2020-01-01 07:00,11.795,0.0,0.0,0.0,0.0,1.0
2020-01-01 07:10,11.704,0.0,0.0,0.0,0.0,1.0
2020-01-01 07:20,11.583,0.0,0.0,0.0,0.0,1.0
2020-01-01 07:30,11.611,0.0,0.0,0.0,0.0,1.0


In [14]:
# # Separate training and testing set 
df_train = df_final[df_final.index<'2021 06 01']


# I'm using all the dataset to train the RNN to improve the performance since ive already
# tried with the testing set and get an accuraccy of 94%
# # Separate training and testing set 
# df_train = df_final[df_final.index<'2021 06 01']

# Define scaler
numerical_scaler2 = MinMaxScaler()
numerical_scaler_out = MinMaxScaler()

numerical_scaler2.fit(df_train);
numerical_scaler_out.fit(df_train.iloc[:,[0,5]])

train_scaled = numerical_scaler2.transform(df_train)

##### Define Timeseries  generators


In [15]:
gen_length = 6 # 1 hour

batchSize = 24*7*2; #  (gen_length//6)  To convert in hour *24 hour * 7 days
# I am predicting column [0,5] i.e ['Cons','Voltage_rise'] even though the only collumn of interest is 5. 
# This is because the RNN does not learn well when one is predicting on a unique feature. 
# Column 5 is the one considered among all because it has the smoothest curve within the backward 
# looking window considered hence yield the best accuracy
train_generator = TimeseriesGenerator(train_scaled, train_scaled[:,[0,5]], 
                                      length = gen_length, 
                                      batch_size= batchSize )

# n_features = train_generator[0][0][0].shape[1]  # Define total number of features
n_features_inputs = 6  # Define total number of features in inputs 
n_features_outputs = 2  # Define total number of features to predicts

#### Define RNN

In [24]:
num_vRise_RNN = Sequential()

num_vRise_RNN.add( LSTM(units=128, activation='relu', input_shape=(gen_length,n_features_inputs)) )
# num_vRise_RNN.add( LSTM(units=128, activation='relu' ) )
num_vRise_RNN.add(Dense(units=n_features_outputs, activation='relu'))

num_vRise_RNN.compile(optimizer='adam', loss='mse', )

##### Define early stopping mechanism

In [26]:
early_stop = EarlyStopping(monitor= 'loss',patience=20, mode='min')

#### Train RNN

In [27]:
num_vRise_RNN.fit(train_generator, 
                  epochs=100, 
                  callbacks=[early_stop], 
                  )

Epoch 1/100
113/113 [==============================] - 2s 10ms/step - loss: 0.0173
Epoch 2/100
113/113 [==============================] - 1s 10ms/step - loss: 0.0037
Epoch 3/100
113/113 [==============================] - 1s 10ms/step - loss: 0.0026
Epoch 4/100
113/113 [==============================] - 1s 10ms/step - loss: 0.0022
Epoch 5/100
113/113 [==============================] - 1s 10ms/step - loss: 0.0020
Epoch 6/100
113/113 [==============================] - 1s 10ms/step - loss: 0.0020
Epoch 7/100
113/113 [==============================] - 1s 10ms/step - loss: 0.0019
Epoch 8/100
113/113 [==============================] - 1s 10ms/step - loss: 0.0019
Epoch 9/100
113/113 [==============================] - 1s 10ms/step - loss: 0.0019
Epoch 10/100
113/113 [==============================] - 1s 11ms/step - loss: 0.0018
Epoch 11/100
113/113 [==============================] - 1s 11ms/step - loss: 0.0017
Epoch 12/100
113/113 [==============================] - 1s 10ms/step - loss: 0.0018
E

In [28]:
num_vRise_RNN.save('pickle_files/RNN/StLaurent_num_vRise_model')

INFO:tensorflow:Assets written to: pickle_files/RNN/StLaurent_num_vRise_model\assets


In [29]:
joblib.dump(numerical_scaler_out,'pickle_files/RNN/StLaurent_num_vRise_scalerPred.plk')
joblib.dump(numerical_scaler2,'pickle_files/RNN/StLaurent_num_vRise_scaler.plk')

['pickle_files/RNN/StLaurent_num_vRise_scaler.plk']